In [3]:
from sklearn import datasets, linear_model, preprocessing, decomposition, manifold
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split

In [ ]:
# We use a digit dataset 10 classes
X, y = datasets.load_digits().data, datasets.load_digits().target

lbl = preprocessing.LabelEncoder()
#only train on samples w label 0,1,2,7,8
y_train = lbl.fit_transform(y[np.where((y == 0) | (y == 1) | (y == 2) | (y == 7) | (y == 8))])
X_train = X[np.where((y == 0) | (y == 1) | (y == 2) | (y == 7) | (y == 8))]

X_train_given,X_test_given,y_train_given,y_test_given = train_test_split(X_train,y_train,test_size=0.2)



# We have Weight matrix, W, d x z
model = linear_model.LogisticRegression(random_state=1)
model.fit(X_train_given, y_train_given)
W = model.coef_.T

#cross validation
print cross_val_score(model, X_train_given, y_train_given, scoring=make_scorer(accuracy_score))

#accuracy only on given samples
s1 = accuracy_score(model.predict(X_test_given),y_test_given)

#accuracy on the samples which aren't given
s2 = accuracy_score(model.predict(X[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 9))]),y[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 9))])
#s2 = accuracy_score(model.predict(X_test_un),y_test_un)

v = s1*len(X_test_given) + s2*len(X[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 9))])
v = v/ (len(X[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 9))])+len(X_test_given))
print(s1,s2,v)

In [41]:
X_train = X_train_given
y_train = y_train_given

In [ ]:
# We have a attributes, a=4 [pca_d1, pca_d2, lle_d1, lle_d2]
# We have Signature matrix, S a x z
pca = decomposition.PCA(n_components=2)
lle = manifold.LocallyLinearEmbedding(n_components=2, random_state=1)
X_pca = pca.fit_transform(X_train)
X_lle = lle.fit_transform(X_train)

for i, ys in enumerate(np.unique(y_train)):
	if i == 0:
		S = np.r_[ np.mean(X_pca[y_train == ys], axis=0), np.mean(X_lle[y_train == ys], axis=0) ]
	else:
		S = np.c_[S, np.r_[ np.mean(X_pca[y_train == ys], axis=0), np.mean(X_lle[y_train == ys], axis=0) ]]

# From W and S calculate V, d x a
V = np.linalg.lstsq(S.T, W.T)[0].T

W_new = np.dot(S.T, V.T).T

print "%f"%np.sum(np.sqrt((W_new-W)**2))

for ys, x in zip(y_train, X_train):
	print np.argmax(np.dot(x.T, W_new)), ys

# INFERENCE
lbl = preprocessing.LabelEncoder()
y_test = lbl.fit_transform(y[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6) | (y == 9))])
X_test = X[np.where((y == 3) | (y == 4) | (y == 5) | (y == 6)| (y == 9))]

# create S' (the Signature matrix for the new classes, using the old transformers)
X_test_un, X_sig, y_test_un, y_sig = train_test_split(X_test, y_test, test_size=5, random_state=1, stratify=y_test)

X_pca = pca.transform(X_sig)
X_lle = lle.transform(X_sig)

for i, ys in enumerate(np.unique(y_sig)):
	if i == 0:
		S = np.r_[ np.mean(X_pca[y_sig == ys], axis=0), np.mean(X_lle[y_sig == ys], axis=0) ]
	else:
		S = np.c_[S, np.r_[ np.mean(X_pca[y_sig == ys], axis=0), np.mean(X_lle[y_sig == ys], axis=0) ]]

# Calculate the new Weight/Coefficient matrix		
W_new = np.dot(S.T, V.T).T

# Check performance
correct = 0
tot = 0
for i, (ys, x) in enumerate(zip(y_test_un, X_test_un)):
	if np.argmax(np.dot(x.T, W_new)) == ys:
		correct +=1
tot +=i
print(correct/float(tot))

corr1 = 0
for i, (ys,x) in enumerate(zip(y_test_given, X_test_given)):
	if np.argmax(np.dot(x.T, W_new)) == ys:
		correct +=1; corr1+=1;

tot+=i
print(corr1/float(i))

print correct/float(tot)